In [2]:
import sage.matroids.matroid
from operator import xor
import functools

In [3]:
t = 1

In [4]:
def iOrder(a,b):
    # a <=_t b, return negative
    # a >=_t b, return positive
    
    if a == b: return 0
    if xor((a < t),(b < t)):
        return -(a-b)
    return a-b

In [5]:
def galeOrder(I,J,order=1):
    # non-comparable, return None
    # I <=_order J under Gale order, return negative
    # I >=_order J, return positive
    # I = J, return 0

    if len(I) == 0 or len(J) == 0:
        return 0
    
    global t
    
    t = order
    
    I.sort(key=functools.cmp_to_key(iOrder))
    J.sort(key=functools.cmp_to_key(iOrder))
    
    flag = iOrder(I[0],J[0])
        
    for j in range(len(I)):
        # compare I,J one by one
        current = iOrder(I[j],J[j])
        
        if current == 0:
            continue #doesn't change whatever we had
        
        if flag != 0 and xor(flag < 0, current < 0):
            return None
        
        flag += current
        
    return flag

In [6]:
def uniformMatroid(n,k):
    bases = [B.list() for B in Subsets(n,k).list()]
    M = Matroid(groundset = list(range(1,n+1)), bases = bases)
    if M.is_valid(): return M
    print("!!!!!WARNING!!!! This matroid somehow is not valid.")

In [7]:
def schubertMatroid(I,n):
#     Bases = {J in [n] choose k : J >=_order I}
#     return a matroid with the bases

    k = len(I)
    bases = []
    
    for J in Subsets(n,k).list():
        J = J.list()
        if galeOrder(I,J) is not None and galeOrder(I,J) <= 0:
            bases.append(J)

    M = Matroid(groundset = list(range(1,n+1)), bases = bases)
    
    if M.is_valid(): return M
    
    print("!!!!!WARNING!!!! This matroid somehow is not valid.")

In [8]:
def LPMatroid(I,J,n):
    # Bases = {B : J >= B >= I}
    
    if len(I) != len(J): print("sth wrong with ur input!!!!")
    
    k = len(I)
    bases = []
    
    for B in Subsets(n,k).list():
        B = B.list()
        if (galeOrder(I,B) is not None and 
            galeOrder(I,B) <= 0 and galeOrder(J,B) >= 0):
            
            bases.append(B)

    M = Matroid(groundset = list(range(1,n+1)), bases = bases)
    
    if M.is_valid(): return M
    

In [141]:
def positroid(u,v):
    # param: u as a list, Gr permutation u in S_n
    # param: v as a list, permutation v in S_n
    # rank = descent index of u
    # bases = {w[k]:v <=w <=u} Bruhat order :permutation class
    
    k = grInfo(u) #rank = descent index
    n = len(u)

    u = Permutation(u)
    v = Permutation(v)
    
    if not v.bruhat_lequal(u):
        print("WARNING!!! ", u, " should be bigger than ", v)
        return
        
    bases = []
    
    for w in SymmetricGroup(n):
        w = Permutation(w)
        if w.bruhat_lequal(u) and v.bruhat_lequal(w):
            current = [w[i] for i in range(k+1)]
            bases.append(current)
#     print(n,bases)
    M = Matroid(groundset = list(range(1,n+1)), bases = bases)
    
    if M.is_valid(): return M

In [123]:
def allFlats(M):
    # param: M, a matroid
    # return: all flats of possible rank
    
    if not M.is_valid():
        print("Your matroid is not valid.")
        return
    
    r = M.full_rank()
    flats = []
    
    for i in range(0,r+1):
        flats += M.flats(i)

    return flats

In [124]:
def quotient(N,M):
    # return true if N is a quotient of M
    
    # choice: the way of determing quotient
    
    Nflats = allFlats(N)
    Mflats = allFlats(M)
    
    for n in Nflats:
        if n not in Mflats:
            return False
    
    return True
    

In [125]:
def grInfo(u,ind=True):
    # if u is not Gr, return False and print eroor
    # if ind=True, return the descent index of u
    # else, return True
    
    flag = 0
    result = 0
    
    for i in range(len(u)-1):
        if u[i] > u[i+1]: 
            flag += 1
            result = i
    
    if ind == True and flag == 1: 
        return result
    if ind == True and flag == 0:
        return len(u)-1
    
    if flag <= 1:
        return True
    
    return False

In [126]:
def matroidInfo(M):
    # print bases, circuits, and flats of matroid M
    
    print("bases of this matroid: \n", [sorted(b) for b in M.bases()])
    print("circuits of this matroid: \n", [sorted(c) for c in M.circuits()])
    print("flats of this matroid: \n",[sorted(f) for f in allFlats(M)])

### Examples

In [127]:
# using Gale order to compare sets

result = galeOrder([1,2,5],[2,4,5],2)
print(result)

# will output 5, which means [1,2,5] >=_2 [2,4,5]

5


In [128]:
#第二个是sage自己有的 用第一种就好 是一样的
M = uniformMatroid(4,2)
M2 = matroids.Uniform(2, 4)
M.is_isomorphic(M2)

True

In [129]:
M = uniformMatroid(4,2)
matroidInfo(M)

bases of this matroid: 
 [[1, 2], [1, 3], [2, 3], [1, 4], [2, 4], [3, 4]]
circuits of this matroid: 
 [[1, 2, 3], [1, 3, 4], [2, 3, 4], [1, 2, 4]]
flats of this matroid: 
 [[], [1], [2], [3], [4], [1, 2, 3, 4]]


In [130]:
# uniform matroid U_{6,3}

M = uniformMatroid(3,2)
M2 = matroids.Uniform(3,6)
M.is_isomorphic(M2)
matroidInfo(M)

bases of this matroid: 
 [[1, 2], [1, 3], [2, 3]]
circuits of this matroid: 
 [[1, 2, 3]]
flats of this matroid: 
 [[], [1], [2], [3], [1, 2, 3]]


In [131]:
M = LPMatroid([1,4],[3,6],6)
matroidInfo(M)

bases of this matroid: 
 [[1, 4], [2, 4], [3, 4], [1, 5], [2, 5], [3, 5], [1, 6], [2, 6], [3, 6]]
circuits of this matroid: 
 [[1, 3], [2, 3], [1, 2], [4, 5], [4, 6], [5, 6]]
flats of this matroid: 
 [[], [1, 2, 3], [4, 5, 6], [1, 2, 3, 4, 5, 6]]


In [132]:
M = LPMatroid([1,3,5],[4,5,6],6)
matroidInfo(M)

bases of this matroid: 
 [[1, 3, 5], [2, 3, 5], [1, 4, 5], [2, 4, 5], [3, 4, 5], [1, 3, 6], [2, 3, 6], [1, 4, 6], [2, 4, 6], [3, 4, 6], [1, 5, 6], [2, 5, 6], [3, 5, 6], [4, 5, 6]]
circuits of this matroid: 
 [[1, 3, 4], [2, 3, 4], [1, 2], [1, 3, 5, 6], [2, 3, 5, 6], [1, 4, 5, 6], [2, 4, 5, 6], [3, 4, 5, 6]]
flats of this matroid: 
 [[], [1, 2], [3], [4], [5], [6], [1, 2, 3, 4], [1, 2, 5], [1, 2, 6], [3, 5], [3, 6], [4, 5], [4, 6], [5, 6], [1, 2, 3, 4, 5, 6]]


In [133]:
M = LPMatroid([1,3],[4,6],6)
matroidInfo(M)

bases of this matroid: 
 [[1, 3], [2, 3], [1, 4], [2, 4], [3, 4], [1, 5], [2, 5], [3, 5], [4, 5], [1, 6], [2, 6], [3, 6], [4, 6]]
circuits of this matroid: 
 [[1, 2], [1, 3, 4], [2, 3, 4], [1, 4, 5], [2, 4, 5], [3, 4, 5], [1, 3, 5], [2, 3, 5], [1, 4, 6], [2, 4, 6], [3, 4, 6], [1, 3, 6], [2, 3, 6], [5, 6]]
flats of this matroid: 
 [[], [1, 2], [3], [4], [5, 6], [1, 2, 3, 4, 5, 6]]


In [134]:
# Schubert matroid SM_[1,3], where [1,3] in [4] choose 2

M = schubertMatroid([1,3],6)
matroidInfo(M)

bases of this matroid: 
 [[1, 3], [2, 3], [1, 4], [2, 4], [3, 4], [1, 5], [2, 5], [3, 5], [4, 5], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6]]
circuits of this matroid: 
 [[1, 2], [1, 3, 4], [2, 3, 4], [1, 4, 5], [2, 4, 5], [3, 4, 5], [1, 3, 5], [2, 3, 5], [1, 5, 6], [2, 5, 6], [3, 5, 6], [4, 5, 6], [1, 4, 6], [2, 4, 6], [3, 4, 6], [1, 3, 6], [2, 3, 6]]
flats of this matroid: 
 [[], [1, 2], [3], [4], [5], [6], [1, 2, 3, 4, 5, 6]]


In [135]:
M = positroid([3,5,6,1,2,4,7],[2,4,1,6,3,5,7])
matroidInfo(M)

2
bases of this matroid: 
 [[1, 2, 4], [1, 3, 4], [1, 2, 5], [1, 3, 5], [2, 4, 6], [3, 4, 6], [2, 5, 6], [3, 5, 6]]
circuits of this matroid: 
 [[2, 3], [7], [4, 5], [1, 6]]
flats of this matroid: 
 [[7], [1, 6, 7], [2, 3, 7], [4, 5, 7], [1, 2, 3, 6, 7], [1, 4, 5, 6, 7], [2, 3, 4, 5, 7], [1, 2, 3, 4, 5, 6, 7]]


### go over all possible positroid

In [172]:
for p in SymmetricGroup(n):
    p = Permutation(p)
    if p.bruhat_lequal(Permutation([1,2,3])):
        print(p)

[1, 2, 3]


In [171]:
n = 3
posiFive = []
for pr in SymmetricGroup(n):
    pr = Permutation(pr)
    if grInfo(pr,ind=False):
        for p in SymmetricGroup(n):
            p = Permutation(p)
            if p.bruhat_lequal(Permutation(pr)):
                M = positroid(pr,p)
                if len(allFlats(M)) != 1:
                    posiFive.append(M)

===
bases of this matroid: 
 [[1, 2, 3]]
circuits of this matroid: 
 []
flats of this matroid: 
 [[], [1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]
[1, 2, 3] [1, 2, 3]
===
bases of this matroid: 
 [[1], [2], [3]]
circuits of this matroid: 
 [[1, 2], [1, 3], [2, 3]]
flats of this matroid: 
 [[], [1, 2, 3]]
[3, 1, 2] [1, 2, 3]
===
bases of this matroid: 
 [[3]]
circuits of this matroid: 
 [[1], [2]]
flats of this matroid: 
 [[1, 2], [1, 2, 3]]
[3, 1, 2] [3, 1, 2]
===
bases of this matroid: 
 [[1], [3]]
circuits of this matroid: 
 [[2], [1, 3]]
flats of this matroid: 
 [[2], [1, 2, 3]]
[3, 1, 2] [1, 3, 2]
===
bases of this matroid: 
 [[2], [3]]
circuits of this matroid: 
 [[1], [2, 3]]
flats of this matroid: 
 [[1], [1, 2, 3]]
[3, 1, 2] [2, 1, 3]
===
bases of this matroid: 
 [[1, 2], [1, 3], [2, 3]]
circuits of this matroid: 
 [[1, 2, 3]]
flats of this matroid: 
 [[], [1], [2], [3], [1, 2, 3]]
[2, 3, 1] [1, 2, 3]
===
bases of this matroid: 
 [[2, 3]]
circuits of this matroid: 
 [[1]]


In [186]:
n = 3
posiFive = []
for pr in SymmetricGroup(n):
    pr = Permutation(pr)
    if grInfo(pr,ind=False):
        for p in SymmetricGroup(n):
            p = Permutation(p)
            if p.bruhat_lequal(Permutation(pr)):
                M = positroid(pr,p)
                F = allFlats(M)
                if len(allFlats(M)) != 1:
                    posiFive.append(M)

In [182]:
for m in posiFive:
    print("===========")
    matroidInfo(m)
#     current = []
#     for q in posiFive:
#         if m != q:
#             if quotient(q,m):
#                 current.append(q)
#     print("------- 's quotients are -------'")
#     for c in current:
#         matroidInfo(c)
#         print("--")

bases of this matroid: 
 [[1, 2, 3, 4, 5]]
circuits of this matroid: 
 []
flats of this matroid: 
 [[], [1], [2], [3], [4], [5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5], [1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5], [1, 4, 5], [2, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 5], [1, 2, 3, 4], [1, 2, 3, 5], [1, 2, 4, 5], [1, 3, 4, 5], [2, 3, 4, 5], [1, 2, 3, 4, 5]]
bases of this matroid: 
 [[1], [2], [3], [4], [5]]
circuits of this matroid: 
 [[1, 2], [1, 3], [2, 3], [1, 4], [2, 4], [3, 4], [1, 5], [2, 5], [3, 5], [4, 5]]
flats of this matroid: 
 [[], [1, 2, 3, 4, 5]]
bases of this matroid: 
 [[5]]
circuits of this matroid: 
 [[1], [2], [3], [4]]
flats of this matroid: 
 [[1, 2, 3, 4], [1, 2, 3, 4, 5]]
bases of this matroid: 
 [[2], [3], [4], [5]]
circuits of this matroid: 
 [[1], [2, 3], [2, 4], [3, 4], [2, 5], [3, 5], [4, 5]]
flats of this matroid: 
 [[1], [1, 2, 3, 4, 5]]
bases of this matroid: 
 [[1], [5]]
circuits of this matroid: 
 [[2], [3],

 [[1, 2, 4], [2, 3, 4]]
circuits of this matroid: 
 [[1, 3], [5]]
flats of this matroid: 
 [[5], [1, 3, 5], [2, 5], [4, 5], [1, 2, 3, 5], [1, 3, 4, 5], [2, 4, 5], [1, 2, 3, 4, 5]]
bases of this matroid: 
 [[1, 2, 3], [1, 3, 4], [2, 3, 4]]
circuits of this matroid: 
 [[5], [1, 2, 4]]
flats of this matroid: 
 [[5], [1, 5], [2, 5], [3, 5], [4, 5], [1, 2, 4, 5], [1, 3, 5], [2, 3, 5], [3, 4, 5], [1, 2, 3, 4, 5]]
bases of this matroid: 
 [[1, 2, 3], [2, 3, 4]]
circuits of this matroid: 
 [[5], [1, 4]]
flats of this matroid: 
 [[5], [1, 4, 5], [2, 5], [3, 5], [1, 2, 4, 5], [1, 3, 4, 5], [2, 3, 5], [1, 2, 3, 4, 5]]
bases of this matroid: 
 [[1, 3, 4], [2, 3, 4]]
circuits of this matroid: 
 [[5], [1, 2]]
flats of this matroid: 
 [[5], [1, 2, 5], [3, 5], [4, 5], [1, 2, 3, 5], [1, 2, 4, 5], [3, 4, 5], [1, 2, 3, 4, 5]]
bases of this matroid: 
 [[2, 3, 4]]
circuits of this matroid: 
 [[1], [5]]
flats of this matroid: 
 [[1, 5], [1, 2, 5], [1, 3, 5], [1, 4, 5], [1, 2, 3, 5], [1, 2, 4, 5], [1, 3, 4, 

In [188]:
for m in posiFive:
#     matroidInfo(m)
    current = []
    for q in posiFive:
        if m != q:
            if quotient(q,m):
                current.append(q)
        
    if len(current) == 0:
        print("===========")
        matroidInfo(m)

bases of this matroid: 
 [[1], [2], [3]]
circuits of this matroid: 
 [[1, 2], [1, 3], [2, 3]]
flats of this matroid: 
 [[], [1, 2, 3]]
bases of this matroid: 
 [[3]]
circuits of this matroid: 
 [[1], [2]]
flats of this matroid: 
 [[1, 2], [1, 2, 3]]
bases of this matroid: 
 [[1], [3]]
circuits of this matroid: 
 [[2], [1, 3]]
flats of this matroid: 
 [[2], [1, 2, 3]]
bases of this matroid: 
 [[2], [3]]
circuits of this matroid: 
 [[1], [2, 3]]
flats of this matroid: 
 [[1], [1, 2, 3]]
bases of this matroid: 
 [[1], [2]]
circuits of this matroid: 
 [[3], [1, 2]]
flats of this matroid: 
 [[3], [1, 2, 3]]
bases of this matroid: 
 [[2]]
circuits of this matroid: 
 [[1], [3]]
flats of this matroid: 
 [[1, 3], [1, 2, 3]]


In [169]:
test = Matroid(bases = [[1]], groundset = [1,2,3])
matroidInfo(test)
# print([sorted(f) for f in test.flats(0)])

bases of this matroid: 
 [[1]]
circuits of this matroid: 
 [[2], [3]]
flats of this matroid: 
 [[2, 3], [1, 2, 3]]
